# Ekerå–Håstad's algorithm for factoring RSA integers
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Ekerå–Håstads algorithm for factoring RSA integers [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20), with improvements from [[E20]](https://doi.org/10.1007/s10623-020-00783-2) and [[E23p]](https://doi.org/10.48550/arXiv.2309.01754), and the post-processing in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).

To start off, let us pick two distinct prime factors $p$ and $q$ uniformly at random from the set of all $l$-bit prime factors, under the restriction that $N = pq$ is a $2l$-bit integer, so as to simulate how the modulus $N$ is typically selected in RSA. To this end, we use the [<code>sample_l_bit_prime(l)</code>](../docs/math/primes/sample_l_bit_prime.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [ ]:
!pip3 install -q quaspy # Make sure that quaspy is installed.

In [1]:
from quaspy.math.primes import sample_l_bit_prime;

l = 1024;

while True:
  p = sample_l_bit_prime(l);

  while True:
    q = sample_l_bit_prime(l);
    if p != q:
      break;

  N = p * q;

  if (2 ** (2 * l - 1)) <= N < (2 ** (2 * l)):
    break;

print("\nSampled p =", p);
print("Sampled q =", q);

print("\nComputed N =", N);


Sampled p = 113236679289835332024713582884343753671887510052584521835089605319788557572947702671040585838675992462948441216244960263331107136903250257698436586110553206943452153094787558166537291533577640678642647893550458427987273369447122707865504063309086709156427642007254080962541907265451975225280711202087433229087
Sampled q = 172270709081841058606569973040467234520311486430323033139361393772331023707962895114523040220708756150282007803819112119215537695720827120994446640545280139256032901126530049098547214559010372319610992918508840416196786743471299095356545799172532324604029663440630897458203565231806449276397938693907921813679

Computed N = 19507363035332958847320596168127456992802417586331450263393772370645782047772283547561881378803262631731614869188550519707196467272190589918233724013168119840724956954205529497014200731999176666259060058190864714145192944200616242944718737505792982282522184616896320465098887376652703951634574119373768802378759781569003768227904962913297004

## 1. Simulating order finding in $\mathbb Z_N^*$ exactly to sample $r$
The first step in the classical pre-processing in Ekerå–Håstad's factoring algorithm [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20) is to select $g$ uniformly at random from $\mathbb Z_N^*$.

[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) for exactly sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and returning its order without explicitly computing and returning $g$. This function requires that the factorization of $N$ is known, as is the case here given that we selected the factors of $N$.

Below, we use this function to simulate the initial classical pre-processing step of Ekerå–Håstad's algorithm. We then setup a simulated generator $g$ for a cyclic group of order $r$ by using the [SimulatedCyclicGroupElement](../docs/math/groups/SimulatedCyclicGroupElement.md) class provided by [Quaspy](https://github.com/ekera/quaspy).

In [2]:
from quaspy.factoring.sampling import sample_r_given_N;

from quaspy.math.groups import SimulatedCyclicGroupElement;

# Sample r.
r = sample_r_given_N(N, factors = [[p, 1], [q, 1]]);

print("Sampled r =", r);

# Setup a simulated cyclic group element of order r.
g = SimulatedCyclicGroupElement(r);

Sampled r = 9753681517666479423660298084063728496401208793165725131696886185322891023886141773780940689401631315865807434594275259853598233636095294959116862006584059920362478477102764748507100365999588333129530029095432357072596472100308121472359368752896491141261092308448160232549443688326351975817287059686884401189237137090316045918636839678686096872316949860627775808077874565395013696447913081347033069740409341962804415036958070017763859551940203724937230020577836334812161387441862951607583007919087040450166052212554393353493567926754773855491119479458573122945324568864567518749748617987132421956103190740742591119154


### 1.1. Simulating the quantum part of Ekerå–Håstad's algorithm
In Ekerå-Håstad's algorithm [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20), with improvements from [[E20]](https://doi.org/10.1007/s10623-020-00783-2) and [[E23p]](
https://doi.org/10.48550/arXiv.2309.01754), the idea is now to use $g$ to compute

$$x = g^{(N - 1) / 2 - 2^{l - 1}} = g^{((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}}$$

classically by using that $N$ is known, and to then compute the discrete logarithm $\log_g x \in [0, r) \cap \mathbb Z$ quantumly.

Let $m = l - 1$ be an upper bound on the bit length of of the the logarithm, and let $\ell = m - \Delta$ for some positive constant $\Delta$.
Let $d = ((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$.
Provided that $r \ge 2^{m+\ell} + (2^\ell - 1)d$, we then have that $d = \log_g x$, in which case we can solve $N = pq$ and $d = ((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$ for $p$ and $q$ by solving a quadratic equation.

In the below example, $x$ is first computed given $N$. Furthermore, $d$ is computed given $p$ and $q$ since $d$ is needed by the simulator for the quantum part of Ekerå–Håstad's algorithm. [Quaspy](https://github.com/ekera/quaspy) provides convenience functions [<code>setup_x_given_g_N(g, N)</code>](../docs/factoring/rsa/setup_x_given_g_N.md) and [<code>setup_d_given_p_q(p, q)</code>](../docs/factoring/rsa/setup_d_given_p_q.md) for computing $x$ and $d$, respectively, but below we perform the computation explicitly to facilitate reader comprehension.

[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) for simulating the quantum part of Ekerå–Håstad's factoring algorithm exactly (up to arbitrary precision) for a given logarithm $d$, order $r$, and parameters $m$, $\ell$ and $\tau$, where $\tau$ is related to the sampling as explained in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). Below, we use the this function to simulate running the quantum part of Ekerå–Håstad's algorithm. The yields a frequency pair $(j, k)$ sampled from the probability distribution induced by the quantum algorithm.

Note that the [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) function checks that $r \ge 2^{m+\ell} + (2^\ell - 1)d$. As explained in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754), this requirement is met with very large probability with $\Delta = 30$.

In [3]:
from quaspy.logarithmfinding.short.sampling import sample_j_k_given_d_r_tau;

# Compute r.
x = g ** ((N - 1) // 2 - 2 ** (l - 1));

# Compute d.
d = ((p - 1) // 2) + ((q - 1) // 2) - 2 ** (l - 1);

# Sanity check that x = g^d.
if x != g ** d:
  raise Exception("Error: Unexpected result.");

# Simulate running the quantum part of the algorithm for g and x.
m = l - 1;
Delta = 30;
ell = m - Delta;
tau = 20;

[j, k] = sample_j_k_given_d_r_tau(d, r, m, ell, tau);

print("Sampled j =", j);
print("Sampled k =", k);

Sampled j = 5811308985316221838550831389328604471508660967884679497339071500246125146632234273395259873032974158465027747154483968473787752447808634020832737129117913959466536982997911940165807356608627975980533110484907734297047947103856341379630586144713607517184930652284076096406732545281839602233439466801425138358893522596573635627188668457707555617158839542014567817856153740407278596434103357274333276120715688468426098862793973907312060000281596375011062493331918388107942887612210160515431915029188509602398126120308650541785504049303638068607366204250796549432364176757917517250950816501384107743163152826963
Sampled k = 53565535137161979043498197374791211275565765801476573639986797414465508200268451188422299555042243102749878560095102697256745145183683781044708455218958814877662895955474459015143170153479722687459816511292146355722918443826667750499043243801367789405043932413937792281083771015560330070152866067152


### 1.2. Solving the frequency pair $(j, k)$ for $p$ and $q$
We now proceed to solve the frequency pair $(j, k)$ for the factors $p$ and $q$ of $N$.

To this end, we first use the [<code>solve_j_k_for_d(j, m, l, g, N, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_j_k_for_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $(j, k)$ for $d$ using the lattice-based post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). We then solve $d$ for the prime factors $p$ and $q$ of $N$ using the post-processing in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754) that is also described in [[E20]](https://doi.org/10.1007/s10623-020-00783-2).

As stated above, to solve $d = ((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$ and $N = pq$ for $p$ and $q$, we only need to solve a quadratic equation.
[Quaspy](https://github.com/ekera/quaspy) provides a convenience function [<code>split_N_given_d(d, N)</code>](../docs/factoring/rsa/postprocessing/split_N_given_d.md) for this purpose, but below we solve explicitly using the quadratic formula to faciliate reader comprehension.

In [4]:
from quaspy.logarithmfinding.short.postprocessing import solve_j_k_for_d;

from gmpy2 import isqrt as sqrt;

# Recover d from the pair (j, k) output by the quantum algorithm.
for [tau, t] in [[4, 17], [7, 17], [14, 17], [17, 17], [20, 19]]:
  recovered_d = solve_j_k_for_d(j, k, m, ell, g, x, tau = tau, t = t);
  if recovered_d != None:
    break;

if None == recovered_d:
  print("[FAIL] Failed to recover d from (j, k).");
else:
  # Form d' = p + q given d.
  p_plus_q = 2 * recovered_d + 2 ** l + 2;

  # Solve d' = p + q and N = pq for p and q using the quadratic formula:
  candidate_p = (p_plus_q - sqrt((p_plus_q ** 2) - 4 * N)) // 2;
  candidate_q = (p_plus_q + sqrt((p_plus_q ** 2) - 4 * N)) // 2;

  if (1 < candidate_p < N) and \
    (1 < candidate_q < N) and \
    (candidate_p * candidate_q == N):
    print("p =", candidate_p);
    print("q =", candidate_q);
    print("\n[ OK ] Successfully recovered p and q.");
  else:
    print("[FAIL] Failed to recover p and q.");

p = 113236679289835332024713582884343753671887510052584521835089605319788557572947702671040585838675992462948441216244960263331107136903250257698436586110553206943452153094787558166537291533577640678642647893550458427987273369447122707865504063309086709156427642007254080962541907265451975225280711202087433229087
q = 172270709081841058606569973040467234520311486430323033139361393772331023707962895114523040220708756150282007803819112119215537695720827120994446640545280139256032901126530049098547214559010372319610992918508840416196786743471299095356545799172532324604029663440630897458203565231806449276397938693907921813679

[ OK ] Successfully recovered p and q.


## 2. Simulating order finding in $\mathbb Z_N^*$ heuristically to sample $g$ and $r$

In addition to the [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) function used above, [Quaspy](https://github.com/ekera/quaspy) also provides a function [<code>sample_g_r_given_N(N, N_factors, ..)</code>](../docs/factoring/sampling/sample_g_r_given_N.md) for sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and heuristically computing and returning its order $r$ alongside $g$.

This function requires that the factorization of $N = pq$ is known, as is the case here given that we selected $p$ and $q$. (If the function is also fed the factorization of $p - 1$ and $q - 1$ then the function can be made exact, as explained in the [documentation](../docs/factoring/sampling/sample_r_given_N.md).)

Below, we use this function to simulate the initial classical pre-processing step of Ekerå–Håstad's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$. This yields $g$ and $r$:

In [5]:
from quaspy.factoring.sampling import sample_g_r_given_N;
from quaspy.math.groups import IntegerModRingMulSubgroupElement;

[g, r] = sample_g_r_given_N(N, [[p, 1], [q, 1]]);

print("Sampled g =", g);
print("Sampled r =", r);

# Setup the group element g.
g = IntegerModRingMulSubgroupElement(g, N);

Sampled g = 1578777241324831297458939604982745578429207277038191604689620186461569690512946048410280919484236949508605611129254489716101953834597125697456109057364426961487715784295924455990431643516354361596736812890767068898320448793163050379820206585868627826270583076968497012976538491225912994626979738521628511644199237568127947456935189615692316933545654832188209823797580503152917847902615832529570421018755414018058350050807542186480254549377107509362214813524800706837539187635410044218271758707676816156259511341247732634950162304911416205190560235647041722137368459616253423791765988378970569081776462748130501776304
Sampled r = 32512271725554931412200993613545761654670695977219083772322953951076303412953805912603135631338771052886024781980917532845327445453650983197056206688613533067874928257009215828357001219998627777098433430318107856908654907001027071574531229176321637137536974361493867441831478961087839919390956865622948003964123790301053486395456132262286989574389832868759252

### 2.1. Simulating the quantum part of Ekerå–Håstad's algorithm
We may then proceed — in analogy with the above — to use the [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to simulate the quantum part of Ekerå–Håstad's factoring algorithm exactly (up to arbitrary precision) for a given logarithm $d$, order $r$, and parameters $m$, $\ell$ and $\tau$, where $\tau$ is related to the sampling as explained in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). This yields a frequency pair $(j, k)$ sampled from the probability distribution induced by the algorithm.

To first setup $x$ and $d$, we use the [<code>setup_x_given_g_N(g, N)</code>](../docs/factoring/rsa/setup_x_given_g_N.md) and [<code>setup_d_given_p_q(p, q)</code>](../docs/factoring/rsa/setup_d_given_p_q.md) convenience functions provided by [Quaspy](https://github.com/ekera/quaspy).

In [6]:
from quaspy.logarithmfinding.short.sampling import sample_j_k_given_d_r_tau;

from quaspy.factoring.rsa import setup_d_given_p_q;
from quaspy.factoring.rsa import setup_x_given_g_N;

# Compute x.
x = setup_x_given_g_N(g, N);

print("Computed x =", x);

# Compute d.
d = setup_d_given_p_q(p, q);

# Sanity check that x = g^d.
if x != g ** d:
  raise Exception("Error: Unexpected result.");

m = l - 1;
Delta = 30;
ell = m - Delta;
tau = 20;

[j, k] = sample_j_k_given_d_r_tau(d, r, m, l = ell, tau = tau);

print("\nSampled j =", j);
print("Sampled k =", k);

Computed x = 6540640774069589946449458584723550624022867369425539175795226853982048119367670676095059398099707884395455582529122444167789168253687944319120859701042226210880480386936528339378059112060817647451197037231515677326872251517918620245320490251897958425824077663183023637826750256564875254399033711685098776977571549478727837287298482446382655038078171526677246969665026956519113401189965779757563990799723991499076519679994761804199959146837817299972034708111302735596815579643933325897762014644092508686382408023814898038539219969039440020795522423923615534671916868213490530623030154980085416558347509717575837056881 (mod 19507363035332958847320596168127456992802417586331450263393772370645782047772283547561881378803262631731614869188550519707196467272190589918233724013168119840724956954205529497014200731999176666259060058190864714145192944200616242944718737505792982282522184616896320465098887376652703951634574119373768802378759781569003768227904962913297004732826098717738459171130

### 2.2. Solving the frequency pair $(j, k)$ for $p$ and $q$
Finally, we may proceed — again in analogy with the above — to solve the frequency pair $(j, k)$ for the factors $p$ and $q$ of $N$.

To this end, we first use the [<code>solve_j_k_for_d(j, m, l, g, N, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_j_k_for_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $(j, k)$ for $d$ using the lattice-based post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). We then solve $d$ and $N$ for the prime factors $p$ and $q$ of $N$ by using the [<code>split_N_given_d(d, N)</code>](../docs/factoring/rsa/postprocessing/split_N_given_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [7]:
from quaspy.logarithmfinding.short.postprocessing import solve_j_k_for_d;

from quaspy.factoring.rsa.postprocessing import split_N_given_d;

# Recover d.
for [tau, t] in [[4, 17], [7, 17], [14, 17], [17, 17], [20, 19]]:
  recovered_d = solve_j_k_for_d(j, k, m, ell, g, x, tau = tau, t = t);
  if recovered_d != None:
    break;

if None == recovered_d:
  print("[FAIL] Failed to recover d from (j, k).");
else:
  result = split_N_given_d(d, N);

  if None != result:
    [recovered_p, recovered_q] = result;

    print("p =", recovered_p);
    print("q =", recovered_q);

    print("\n[ OK ] Successfully recovered p and q.");
  else:
    print("[FAIL] Failed to recover p and q.");

p = 172270709081841058606569973040467234520311486430323033139361393772331023707962895114523040220708756150282007803819112119215537695720827120994446640545280139256032901126530049098547214559010372319610992918508840416196786743471299095356545799172532324604029663440630897458203565231806449276397938693907921813679
q = 113236679289835332024713582884343753671887510052584521835089605319788557572947702671040585838675992462948441216244960263331107136903250257698436586110553206943452153094787558166537291533577640678642647893550458427987273369447122707865504063309086709156427642007254080962541907265451975225280711202087433229087

[ OK ] Successfully recovered p and q.
